# Linear Regression vs. RANSAC

## This notebook is used for post-processing of individual planes and allows you to compare the effect of Linear Regression and RANSAC

### The following libraries are used: numpy, open3d, scikit learn

# Common step

In [1]:
import numpy as np
import open3d as o3d
from sklearn import linear_model

pcd=o3d.io.read_point_cloud("plane2.pcd")

# visualization
o3d.visualization.draw_geometries([pcd])


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Linear Regression

### This part is devoted to finding the approximate plane using the linear regression method

#### The result of executing the code below is a point cloud containing the original plane and found by Linear Regression

In [2]:
X_lr=np.asarray(pcd.points)
X_lr_colors=np.asarray(pcd.colors)
y_lr=np.negative(np.ones((X_lr.shape[0],1)))

In [3]:
lr=linear_model.LinearRegression(fit_intercept=False)
lr.fit(X_lr,y_lr)

#print(lr.predict(X_lr))
#print(lr.coef_)

LinearRegression(fit_intercept=False)

In [4]:
#ax+by+cz+d=0 (Ax+By+Cz=-1)
color_new=np.random.sample([3])
for p in range(X_lr.shape[0]):
    z=-1-(lr.coef_[0][0]*X_lr[p][0])-(lr.coef_[0][1]*X_lr[p][1])
    coord=np.hstack((X_lr[p][:2], np.array(z)))
    X_lr=np.append(X_lr, [coord], axis=0)
    X_lr_colors=np.append(X_lr_colors, [color_new], axis=0)

In [5]:
pcd_lr=o3d.geometry.PointCloud()
pcd_lr.points=o3d.utility.Vector3dVector(X_lr)
pcd_lr.colors=o3d.utility.Vector3dVector(X_lr_colors)
o3d.io.write_point_cloud("plane2lr.pcd", pcd_lr)

True

In [6]:
pcd_3=o3d.io.read_point_cloud("plane2lr.pcd")

# visualization
o3d.visualization.draw_geometries([pcd_3])

# RANSAC

### This part is devoted to finding the approximate plane using the RANSAC method

#### The result of executing the code below is a point cloud containing the original plane in which the outliers are painted with a different color


In [7]:
# Convert Open3D.o3d.geometry.PointCloud to numpy array
X=np.asarray(pcd.points)
y=np.negative(np.ones((X.shape[0],1)))

In [8]:
ransac=linear_model.RANSACRegressor(residual_threshold=0.003, max_trials=200)
ransac.set_params(**{'base_estimator':linear_model.LinearRegression(fit_intercept=False)})
ransac.fit(X,y)

#print(ransac.predict(X))

RANSACRegressor(base_estimator=LinearRegression(fit_intercept=False),
                max_trials=200, residual_threshold=0.003)

In [9]:
inlier_mask=ransac.inlier_mask_

#print(ransac.estimator_.coef_)

In [10]:
#counting outliers
sum=0
for i in inlier_mask:
    if not i:
        sum+=1
print(sum)

313


In [11]:
color=np.random.sample([3])
X_colors=np.asarray(pcd.colors)

In [12]:
out_color=np.random.sample([3])
for point in range(X.shape[0]):
    if not inlier_mask[point]:
        X_colors[point]=out_color

In [13]:
pcd_ransac=o3d.geometry.PointCloud()
pcd_ransac.points=o3d.utility.Vector3dVector(X)
pcd_ransac.colors=o3d.utility.Vector3dVector(X_colors)
o3d.io.write_point_cloud("plane2ransac.pcd", pcd_ransac)

True

In [14]:
pcd_2=o3d.io.read_point_cloud("plane2ransac.pcd")

# visualization
o3d.visualization.draw_geometries([pcd_2])